In [50]:
import sys
import json
import pandas as pd
from pandas.io.json import json_normalize
sys.path.append("../") # Enables importing from parent directory
import fpl_draft_league as fpl

### Transactions Json

In [51]:
# Pull the transactions API data into json
fpl.get_json('transactions.json',
         'https://draft.premierleague.com/api/draft/league/38996/transactions',
        'lee.gower17@gmail.com') 

Enter Password:  ········


In [53]:
with open('transactions.json') as json_data:
        d = json.load(json_data)
        transactions_df = json_normalize(d['transactions'])

### Elements json

In [54]:
fpl.get_json('elements.json',
            'https://draft.premierleague.com/api/bootstrap-static',
            'lee.gower17@gmail.com')

Enter Password:  ········


In [55]:
with open('elements.json') as json_data:
        d = json.load(json_data)
        elements_df = json_normalize(d['elements'])

### Details json

In [56]:
fpl.get_json('details.json',
            'https://draft.premierleague.com/api/league/38996/details',
            'lee.gower17@gmail.com')

Enter Password:  ········


In [57]:
with open('details.json') as json_data:
        d = json.load(json_data)
        entries_df = json_normalize(d['league_entries'])

## Total transactions by player
I've got the transactions_df, and can get the league_entries_df, so just need to join these to and then do some aggregation.

In [58]:
entries_df = entries_df[['entry_id', 'player_first_name']]
elements_df = elements_df[['id', 'first_name', 'second_name']]

In [68]:
df = pd.merge(transactions_df,
              entries_df,
              how='left',
              left_on='entry',
              right_on='entry_id')

In [69]:
df = pd.merge(df,
              elements_df,
              how='left',
              left_on='element_in',
              right_on='id')

In [70]:
df = pd.merge(df,
             elements_df,
             how='left',
             left_on='element_out',
             right_on='id')

In [72]:
df['player_in'] = df['first_name_x'] + ' ' + df['second_name_x']
df['player_out'] = df['first_name_y'] + ' ' + df['second_name_y']

In [75]:
df = df.drop(['first_name_x',
         'first_name_y',
         'second_name_x',
         'second_name_y',
         'id', 'id_y',
         'entry_id',
         'entry',
         'element_in',
         'element_out',
         'id_x',], axis=1)

In [82]:
df[df.player_first_name == 'Liam']

,added,event,index,kind,priority,result,player_first_name,player_in,player_out
23,2019-08-16T07:20:15.761884Z,2,1.0,w,1.0,a,Liam,Nicolas Pépé,Ilkay Gündogan
24,2019-08-16T07:20:15.762005Z,2,16.0,w,2.0,di,Liam,Nicolas Pépé,Abdoulaye Doucouré
25,2019-08-16T07:20:15.762091Z,2,17.0,w,3.0,di,Liam,Giovani Lo Celso,Ilkay Gündogan
26,2019-08-16T07:20:15.762175Z,2,18.0,w,4.0,di,Liam,Giovani Lo Celso,Abdoulaye Doucouré
27,2019-08-16T07:20:15.762257Z,2,19.0,w,5.0,di,Liam,Moise Kean,Gerard Deulofeu
28,2019-08-16T07:20:15.762339Z,2,20.0,w,6.0,a,Liam,Nick Pope,Ben Foster
29,2019-08-16T07:20:15.762421Z,2,33.0,w,7.0,di,Liam,Daniel James,Ilkay Gündogan
30,2019-08-16T07:20:15.762503Z,2,34.0,w,8.0,di,Liam,Daniel James,Abdoulaye Doucouré
31,2019-08-16T07:20:15.762584Z,2,35.0,w,9.0,do,Liam,Phil Foden,Ilkay Gündogan
32,2019-08-16T07:20:15.762665Z,2,36.0,w,10.0,a,Liam,Phil Foden,Abdoulaye Doucouré


In [44]:
df[df['player_first_name'] == 'Liam']

,added,element_in,element_out,entry,event,id,index,kind,priority,result,entry_id,player_first_name
23,2019-08-16T07:20:15.761884Z,487,223,174687,2,2559556,1.0,w,1.0,a,174687,Liam
28,2019-08-16T07:20:15.762339Z,92,367,174687,2,2559561,20.0,w,6.0,a,174687,Liam
32,2019-08-16T07:20:15.762665Z,221,369,174687,2,2559565,36.0,w,10.0,a,174687,Liam
110,2019-08-27T10:25:33.248992Z,504,221,174687,4,4570798,2.0,w,1.0,a,174687,Liam
112,2019-08-27T10:25:33.249184Z,31,75,174687,4,4570800,15.0,w,3.0,a,174687,Liam
344,2019-12-13T11:17:22.291472Z,161,58,174687,17,13848029,7.0,w,2.0,a,174687,Liam
357,2019-12-24T12:54:32.132056Z,303,31,174687,19,14767327,2.0,w,1.0,a,174687,Liam
